In [1]:
import pandas as pd
import datetime
from quant_utils.constant import DB_CONN_LOCAL_MYSQL

In [88]:
df = pd.read_excel("D:/分钟行情/159915.SZ.xlsx")
df["日期"] = df["日期"].apply(lambda s: s + datetime.timedelta(minutes=1))
df['成交额(百万)'] = df['成交额(百万)'] * 1000000
condition_1 = (df["日期"].dt.hour == 11) & (df["日期"].dt.minute == 31)
condition_2 = (df["日期"].dt.hour == 15) & (df["日期"].dt.minute == 1)     
for idx in range(0, len(df)):
    if condition_1[idx] or condition_2[idx]:
        df.loc[idx-1, "成交额(百万)"] = df.loc[idx-1, "成交额(百万)"] + df.loc[idx, "成交额(百万)"]
        df.loc[idx-1, "成交量"] = df.loc[idx-1, "成交量"] + df.loc[idx, "成交量"]
        df.loc[idx-1, "收盘价(元)"] = df.loc[idx, "收盘价(元)"]
        print(df.loc[idx, "日期"], df.loc[idx-1, "成交量"])
result_df = df[~(condition_1|condition_2)]

result_df.rename(
    columns={
        "代码": 'code',
        "日期": "time",
        "开盘价(元)": "open",
        "最高价(元)": "high",
        "最低价(元)": "low",
        "收盘价(元)": "close",
        "成交额(百万)": "total_amount",
        "成交量": "turnover",
    },
    inplace=True
)
result_df.drop(columns=["名称", "涨跌幅"], inplace=True)
result_df["date"] = result_df["time"].dt.date
result_df["tick"] = result_df["time"].dt.time
result_df["tick"] = result_df["tick"].apply(lambda s: s.strftime("%H%M%S"))

2023-05-17 15:01:00 7520100
2023-05-18 11:31:00 2322300
2023-05-18 15:01:00 6138600
2023-05-19 11:31:00 2533700
2023-05-19 15:01:00 6981300
2023-05-22 11:31:00 1253800
2023-05-22 15:01:00 4814600
2023-05-23 11:31:00 404500
2023-05-23 15:01:00 9898900
2023-05-24 11:31:00 2071100
2023-05-24 15:01:00 5028600
2023-05-25 11:31:00 4680200
2023-05-25 15:01:00 6514500
2023-05-26 11:31:00 1973000
2023-05-26 15:01:00 4764200
2023-05-29 11:31:00 749300
2023-05-29 15:01:00 6038300
2023-05-30 11:31:00 1692100
2023-05-30 15:01:00 10057700
2023-05-31 11:31:00 3619800
2023-05-31 15:01:00 14337100
2023-06-01 11:31:00 959000
2023-06-01 15:01:00 6885900
2023-06-02 11:31:00 1619200
2023-06-02 15:01:00 9190700
2023-06-05 11:31:00 1740300
2023-06-05 15:01:00 4879800
2023-06-06 11:31:00 654800
2023-06-06 15:01:00 12090800
2023-06-07 11:31:00 1422600
2023-06-07 15:01:00 10091100
2023-06-08 11:31:00 2073100
2023-06-08 15:01:00 10303500
2023-06-09 11:31:00 568400
2023-06-09 15:01:00 9039200
2023-06-12 11:31:00 

C:\Users\Wilcoxon\AppData\Local\Temp\ipykernel_11548\2538446297.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df.rename(
C:\Users\Wilcoxon\AppData\Local\Temp\ipykernel_11548\2538446297.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df.drop(columns=["名称", "涨跌幅"], inplace=True)
C:\Users\Wilcoxon\AppData\Local\Temp\ipykernel_11548\2538446297.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing

In [91]:
result_df = result_df.set_index("time")

KeyError: "None of ['time'] are in the columns"

In [93]:
result_df.resample('30min', closed='right', label='right').agg(
    {
        'open': 'first', 
        'high': 'max', 
        'low' :'min', 
        'close':'last',
        "total_amount": 'sum',
        'turnover': 'sum'
    }
    ).dropna()

,open,high,low,close,total_amount,turnover
time,,,,,,
2023-05-17 14:00:00,2.219,2.221,2.219,2.221,3640000.0,1640700
2023-05-17 14:30:00,2.220,2.224,2.219,2.223,87530000.0,39388400
2023-05-17 15:00:00,2.222,2.230,2.221,2.229,303690000.0,136422273
2023-05-18 10:00:00,2.233,2.237,2.221,2.222,257070000.0,115299800
2023-05-18 10:30:00,2.222,2.231,2.222,2.226,114560000.0,51436478
...,...,...,...,...,...,...
2023-11-07 11:00:00,1.965,1.966,1.960,1.965,124530000.0,63444755
2023-11-07 11:30:00,1.965,1.965,1.960,1.961,85860000.0,43750900
2023-11-07 13:30:00,1.961,1.967,1.960,1.965,55710000.0,28378200


In [123]:
def get_1m_quotes(code, start_time=None, end_time=None):
    query_sql = f"""
        select
            time,
            open,
            high,
            low,
            close,
            total_amount as total_amount,
            turnover as turnover
        from 
            {code}
        where 1=1
    """
    if start_time:
        query_sql += f' and time >= "{start_time}"'
    if end_time:
        query_sql += f' and time <= "{end_time}"'

    df = DB_CONN_LOCAL_MYSQL.exec_query(query_sql).set_index('time')
    return df
df = get_1m_quotes("SH000300")
df.resample('D').agg(
    {
        'open': 'first', 
        'high': 'max', 
        'low' :'min', 
        'close':'last',
        "total_amount": 'sum',
        'turnover': 'sum'
    }).dropna()

,open,high,low,close,total_amount,turnover
time,,,,,,
2023-10-31,3581.6759,3581.6759,3554.5822,3574.5368,1.928891e+11,1.109528e+10
2023-11-01,3605.7576,3606.4126,3564.6421,3572.9675,1.811460e+11,9.661952e+09
2023-11-02,3580.0672,3597.1699,3553.7406,3555.2441,1.479358e+11,8.950171e+09
2023-11-03,3558.5303,3596.1010,3558.5303,3584.1465,1.639313e+11,9.640022e+09
2023-11-06,3608.3187,3635.7897,3600.6304,3632.3903,2.519888e+11,1.514479e+10
2023-11-07,3623.7849,3628.4615,3606.2975,3620.8806,1.884245e+11,1.090890e+10


In [2]:
df = pd.read_csv("d:/分钟行情/159901-1min.csv")

In [4]:
df=df.set_index("trade_time")
df.index = pd.to_datetime(df.index)

In [ ]:
df_1d = df.resample('D').agg(
    {
        'open': 'first', 
        'high': 'max', 
        'low' :'min', 
        'close':'last',
        "amount": 'sum',
        'vol': 'sum'
    }
).dropna()
df_1d

In [19]:
df_30min = df.resample('30min', closed='right', label='right').agg(
    {
        'open': 'first', 
        'high': 'max', 
        'low' :'min', 
        'close':'last',
        "amount": 'sum',
        'vol': 'sum'
    }
).dropna()

In [27]:
idx = list(range(0, len(df_30min)//2))*2
idx.sort()
df_30min["idx"] = idx
df_30min = df_30min.reset_index()
df_60min = df_30min.groupby(by="idx").agg(
    {
        "trade_time": 'last',
        'open': 'first', 
        'high': 'max', 
        'low' :'min', 
        'close':'last',
        "amount": 'sum',
        'vol': 'sum'
    }
)